In [2]:
from tensorflow import keras as ks
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



One-hotted Data and Models



In [3]:
one_hot_data = pd.read_csv('one_hotted_training_data.csv')
one_hot_data = one_hot_data[one_hot_data['price'] <= 50000]
one_hot_data = one_hot_data[one_hot_data['price'] > 500]
#print(one_hot_data.price.describe())
print(one_hot_data)

train, test = train_test_split(one_hot_data, test_size=0.2)
X_train = train.drop('price', axis=1).to_numpy()
y_train = train['price'].to_numpy()
X_valid = test.drop('price', axis=1).to_numpy()
y_valid = test['price'].to_numpy()


         price  year  condition  cylinders  odometer  posting_date  \
0      15000.0  2013         10          6    128000          2021   
1      19900.0  2004          6          8     88000          2021   
2      14000.0  2012         10          6     95000          2021   
3      22500.0  2001          6          8    144700          2021   
4      15000.0  2017         10          8     90000          2021   
...        ...   ...        ...        ...       ...           ...   
65798  18000.0  2005         10          8    130000          2021   
65799  18000.0  1990          6          8    114400          2021   
65800   9800.0  1985         15          6    115000          2021   
65801   6800.0  1997          6          8     69550          2021   
65802  21900.0  1920          6          6     11065          2021   

       income_this_year  manufactureracura  manufactureraudi  manufacturerbmw  \
0                 49769                  0                 0                0 

In [4]:
print("X_train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)
print("X_valid Shape:", X_valid.shape)
print("y_valid Shape:", y_valid.shape)

X_train Shape: (51132, 64)
y_train Shape: (51132,)
X_valid Shape: (12784, 64)
y_valid Shape: (12784,)


In [5]:
model_in = ks.Input(shape=(X_train.shape[1], ), dtype='float32')

out = ks.layers.LayerNormalization()(model_in)
out = ks.layers.Dense(1024, activation='relu')(out)
out = ks.layers.Dense(1024, activation='relu')(out)
out = ks.layers.Dropout(0.2)(out)
out = ks.layers.Dense(1024, activation='relu')(out)
out = ks.layers.Dropout(0.2)(out)
out = ks.layers.Dense(512, activation='relu')(out)
out = ks.layers.Dropout(0.2)(out)
out = ks.layers.Dense(512, activation='relu')(out)
out = ks.layers.Dropout(0.2)(out)
out = ks.layers.Dense(256, activation='relu')(out)
out = ks.layers.Dropout(0.2)(out)
out = ks.layers.Dense(128, activation='relu')(out)
out = ks.layers.Dense(64, activation='relu')(out)
out = ks.layers.Dense(32, activation='relu')(out)
out = ks.layers.Dense(16, activation='relu')(out)
out = ks.layers.Dense(1)(out)

model = ks.Model(model_in, out)
#model.summary()
model.compile(loss='mean_squared_error', optimizer=ks.optimizers.Adam(lr=3e-5))

Metal device set to: Apple M1 Pro


2022-12-23 00:48:00.766808: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-23 00:48:00.767475: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/anaconda3/envs/keras_test_env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
model.fit(x=X_train, y=y_train, batch_size=100, epochs=15, verbose=1, validation_data=(X_valid, y_valid))

Epoch 1/15


2022-12-23 00:49:11.393713: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-23 00:49:11.825081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


512/512 [==============================] - ETA: 0s - loss: 135576576.0000

2022-12-23 00:49:18.140664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


512/512 [==============================] - 7s 12ms/step - loss: 135576576.0000 - val_loss: 81554648.0000
Epoch 2/15
512/512 [==============================] - 5s 10ms/step - loss: 81012272.0000 - val_loss: 81713472.0000
Epoch 3/15
512/512 [==============================] - 5s 10ms/step - loss: 80753800.0000 - val_loss: 81750120.0000
Epoch 4/15
512/512 [==============================] - 5s 10ms/step - loss: 80577832.0000 - val_loss: 81468936.0000
Epoch 5/15
512/512 [==============================] - 5s 10ms/step - loss: 80472512.0000 - val_loss: 80796808.0000
Epoch 6/15
512/512 [==============================] - 5s 10ms/step - loss: 80382336.0000 - val_loss: 80767304.0000
Epoch 7/15
512/512 [==============================] - 5s 10ms/step - loss: 80316464.0000 - val_loss: 80835056.0000
Epoch 8/15
512/512 [==============================] - 5s 10ms/step - loss: 80219736.0000 - val_loss: 80559760.0000
Epoch 9/15
512/512 [==============================] - 5s 10ms/step - loss: 80183664.0000 -

In [7]:
preds = model.predict(X_valid)
for i in range(5):
    print(preds[i], y_valid[i])
        
model.save("NN_model.h5")

 55/400 [===>..........................] - ETA: 0s

2022-12-23 00:52:14.535339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


400/400 [==============================] - 1s 3ms/step
[8323.764] 6500.0
[12107.316] 14995.0
[12319.007] 14950.0
[14476.541] 23500.0
[12538.393] 12000.0


In [ ]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

# Predict the response for test dataset
preds = clf.predict(X_valid)

import joblib
# Export model

# Basically, this model is too large to upload in github, but trains extremely quickly.
# Thus, in the app.py file, the model is just trained every time the app starts up instead of storing the model
# in permanent storage
joblib.dump(clf, "decision_tree.pkl") 

